## Combining predicted probabilities from several classifiers, for each ligand

In [1]:
import pandas as pd
import numpy as np
import pickle
from collections import defaultdict
from os import environ

In [4]:
curr_dir = !pwd
#models = ["XGB", "RF", "ADA", "KNN", "SVM", "Logistic"]
models = ["XGB", "RF", "ADA", "SVM", "KNN", "Logistic"]
#ligands = ["dna", "dnabase", "dnabackbone", "rna", "rnabase", "rnabackbone", "peptide", "ion", "metabolite"]
ligands = ["peptide", "ion", "metabolite"]
dataset_path = curr_dir[0]+"/1st_level_pred/"
between_path = curr_dir[0]+"/../pred_AUC_AUPRC/mediode_NegLigand_NoFilter/NoDown/01.25.2018/between_pred_tables/"
K = 10

In [5]:
%%time
pred_prob_dict = defaultdict(dict)

for ligand in ligands:
    #Read the data tables
    for model in models:
        pred_prob_dict[ligand][model] = dict()
        
        #Reading the ligand-model between labels positions
        between_table = pd.read_csv(between_path+ligand+"_"+model+"_features_pred.csv", sep='\t', index_col=0)[["fold", "prob"]]
        between_table.columns = ["fold", model+"_prob"]
        
        for fold in range(1,K+1):
            pred_prob_dict[ligand][model][fold] = dict()
            #Reading the ligand regular dataset
            pred_prob_dict[ligand][model][fold]["dataset"] = pd.read_csv(dataset_path+str(fold)+"/"+ligand+"_"+model+"_fold"+str(fold)+"_features_pred.csv", sep='\t', index_col=0)[["fold", "prob"]]
            pred_prob_dict[ligand][model][fold]["dataset"].columns = ["fold", model+"_prob"]
            pred_prob_dict[ligand][model][fold]["between"] = between_table[between_table["fold"] == fold]
        print "Finished reading "+ligand+" "+model
    print "Finished reading data "+ligand
    
    #Combine them into ligand folds probs table
    for fold in range(1,K+1):
        
        probs_merged_df = pd.DataFrame()
        between_merged_df = pd.DataFrame()
        
        for model in models:
        
            #The regular dataset
            if (probs_merged_df.shape[0] == 0):
                probs_merged_df = pred_prob_dict[ligand][model][fold]["dataset"]
            else:
                probs_merged_df = probs_merged_df.merge(pred_prob_dict[ligand][model][fold]["dataset"], on="fold", left_index=True, right_index=True)
            #The positions between labels
            if (between_merged_df.shape[0] == 0):
                between_merged_df = pred_prob_dict[ligand][model][fold]["between"]
            else:
                between_merged_df = pd.concat([between_merged_df, pred_prob_dict[ligand][model][fold]["between"][[model+"_prob"]]], axis=1)
    
        #Saving to .csv
        probs_merged_df.to_csv(dataset_path+str(fold)+"/"+ligand+"_dataset_probs.csv", sep='\t')
        between_merged_df.to_csv(dataset_path+str(fold)+"/"+ligand+"_between_probs.csv", sep='\t')
    
    print "Finished writing "+ligand

Finished reading peptide XGB
Finished reading peptide RF
Finished reading peptide ADA
Finished reading peptide SVM
Finished reading peptide KNN
Finished reading peptide Logistic
Finished reading data peptide
Finished writing peptide
Finished reading ion XGB
Finished reading ion RF
Finished reading ion ADA
Finished reading ion SVM
Finished reading ion KNN
Finished reading ion Logistic
Finished reading data ion
Finished writing ion
Finished reading metabolite XGB
Finished reading metabolite RF
Finished reading metabolite ADA
Finished reading metabolite SVM
Finished reading metabolite KNN
Finished reading metabolite Logistic
Finished reading data metabolite
Finished writing metabolite
CPU times: user 8min 42s, sys: 46.5 s, total: 9min 28s
Wall time: 10min 46s
